In [1]:
# 050224
# testing dataloader

In [2]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output



import pprint


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor,IDSWDataSetLoader
from architectures import sevennet, smallnet1, smallnet2, smallnet3, build_net
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, train_model, train_log, log_test_score, set_lossfn, pipeline


from fns4wandb import hp_sweep_DL
import pickle
import torch.optim.lr_scheduler as lr_scheduler
device = "cuda:1" if torch.cuda.is_available() else "cpu"
from fns4wandb import train
from datetime import date

from torchvision.models import vgg16

In [3]:

def train_model(model,  x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 

    
    
    loss_fn = set_lossfn(config['loss_fn']) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['scheduler'], last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
                                                                    #scheduler'
    ####
    
    #model = model.to(device)
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []

    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        if epoch ==0:
            best_model=model
            best_acc=0

        t_loss, train_prediction, t_correct, model, optimizer = loop(model=model, X=x_train, Y=y_train, loss_fn=loss_fn, device=device, col_dict=col_dict, num_classes=11, model_name= config['model_name'], optimizer=optimizer, scheduler =scheduler)
        
        
        save_dict['t_loss_list'] = t_loss_list
        save_dict['t_labels'] = y_train
        save_dict['t_predict_list'] = t_predict_list 

        train_acc = (t_correct / len(x_train))
        t_accuracy_list.append(train_acc)
        
        save_dict['t_accuracy_list'] = t_accuracy_list 
        clear_output()
        
            
        print('validating...')
        
        v_loss, val_prediction, val_correct= loop(model=model, X=x_val, Y=y_val, loss_fn=loss_fn, device=device, col_dict=col_dict, num_classes=11, model_name= config['model_name'], train=False)


        save_dict['v_loss_list'] = v_loss_list
        save_dict['v_predict_list'] = v_predict_list  #
        save_dict['v_labels'] = y_val
        
        val_acc = (t_correct / len(x_val))
        v_accuracy_list.append(val_acc)
        save_dict['v_accuracy_list'] = v_accuracy_list  #
        

        clear_output()
            
        total_epochs += epoch

        if val_acc > best_acc:

            best_acc = val_acc

            best_model = model#deepcopy(model)
            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['training_samples'] = len(x_train)
            save_dict['validation_samples'] = len(x_val)
            

            title = save_dict['Run']
            save_location = save_dict['save_location']
            conf= dict(config)
            diction = {}
            d = date.today()
            d=str(d)
            diction.update({'Date':d})
            diction.update(save_dict)
            diction.update(conf)
            diction.update(col_dict)
            print(diction)
            
            save2json(conf, title, save_location)
            save2csv(diction, title, save_location)
            
            diction['model.state_dict'] = model.state_dict() #to('cpu').
            
            with open(f"{save_location}{title}.pkl", 'wb+') as f:
                pickle.dump(diction, f)
            
            
            
            print('improvment in metrics. model saved')

            #print('END Of SAve -----------------------------')
            #!nvidia-smi

        


            
    model = best_model

    return model



def pipeline(config, save_dict, col_dict): 
    # set model
    l = config['f_lin_lay']
    
    
    # load in data
    file_path =  r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'
    random_seed =1
    img_len = len(os.listdir(file_path))


    #print(ids[4])
    x, y = import_imagedata(file_path)

    x_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,
                                     random_state=random_seed, shuffle=True)
    x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,
                                     random_state=random_seed, shuffle=True)

    x_train, x_val = x_train[:2], x_val[:2] # for testing purposes, shortened data
    #!nvidia-smi
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    #loss_fn = nn.MSELoss()
    title=save_dict['Run']
    
    """l = config['f_lin_lay']
    model_vgg16 = vgg16(weights="IMAGENET1K_V1")
    vgg_feats = model_vgg16.features
    vgg_classifier = model_vgg16.classifier
    #print(vgg_classifier)
    vgg_classifier.pop(6)
    vgg = nn.Sequential(
        vgg_feats,
        Flattern(),
        vgg_classifier,
        nn.Linear(l,11),
        nn.Softmax(dim=0),
        )
    """

    model =sevennet(in_chan=3, f_lin_lay=config['f_lin_lay'], l_lin_lay=11, ks=config['ks'], dropout = config['dropout']).to(device)
    #model = vgg
    model = train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl

    return model


In [8]:

config = {
    "model_name" : '7c3l',
    "epochs" :2, 
    "learning_rate" : 5.97E-05, #6.62E-05, #5.97E-05, #6.01E-05, #6.62E-05, #0.00, 00821591686076769, #8e-5,
    "dataset" : 'IDSW_Aug',
    "architecture" :'CNN',
    "optimizer": 'adam',
    "loss_fn" : 'CrossEntropy',
    "weight_decay": 2e-5, #4e-5, #2e-5, #3.00E-05,
    "dropout" : 0.4, #0.4,
    "lin_layer_size": 100,
    "ks" : [3,5],
    "in_chan" : 3,
    "num_classes" :11,
    "scheduler" : 0.2,
    "channels": 3,
    "f_lin_lay" :172032
    , 
     
}




#r'/its/home/nn268/antvis/optics/
#pickles
col_dict = {
    'colour': 'colour',
    'size': [15, 5],
    'pad': 0,
    'model_size': '2c2l'
}


save_dict = {'Run' : 'testin 7c small res blank padding',
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}




In [9]:
class Flattern(nn.Module):
    def __init__(self):
        super(Flattern, self).__init__()
    def forward(self, x):
        #print(x.shape)
        x = x.flatten()
        return x

model = pipeline(config, save_dict, col_dict)

100%|████████████| 2/2 [00:00<00:00, 60.55it/s]
